# CamelsOptimizer - find the ultimate person to trade for

## Install dependencies

In [ ]:
!pip install requests hyperopt

In [13]:
import requests
import re
import time
from itertools import product
from hyperopt import hp, tpe, fmin, Trials
import numpy as np

## Prepare requests

In [14]:
# Headers for request to server
headers = {
            'authority': 'kamelrechner.eu',
            'method': 'POST',
            'path': '/en/result',
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-encoding': 'gzip, deflate',
            'accept-language': 'he,en-US;q=0.9,en;q=0.8,he-IL;q=0.7',
            'cache-control': 'max-age=0',
            'content-length': '93',
            'content-type': 'application/x-www-form-urlencoded',
            'cookie': '__cfduid=d6242d07e762c2bbf4dacfea3643648aa1582985884; _ga=GA1.2.1196108495.1582985885; _gid=GA1.2.734670141.1582985885; _gat_gtag_UA_423721_8=1',
            'origin': 'https://kamelrechner.eu',
            'referer': 'https://kamelrechner.eu/en/male',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36'
}

In [5]:
parameters = {
    'age': list(range(14, 71)),
    'height': list(range(140, 221)),
    'haircolor': ['blonde', 'brown', 'black', 'red', 'gray'],
    'hair': ['long', 'middle', 'short',' bald'],
    'eyecolor': ['blue', 'green', 'brown', 'grey'],
    'beard': ['none', 'small', 'middle', 'large'],
    'body': ['muscle', 'normal', 'chubby']
}

## Utility functions for fetching results from server

In [3]:
def generate_request(age, height, haircolor, hair, eyecolor, beard, body):
    data = f'age={age}&height={height}&haircolor={haircolor}&hair={hair}&eyecolor={eyecolor}&beard={beard}&body={body}&gender=male'
    return requests.post('https://kamelrechner.eu/en/result', headers=headers, data=data, verify=True)

In [4]:
def parse_request(request):
    match = re.search('<span class="result">(\d+)</span>', request.text)
    result = int(match.group(1))
    return result

In [6]:
def generate_perms(options_dict):
    keys, values = zip(*options_dict.items())
    for perm in product(*values):
        yield dict(zip(keys, perm))

In [7]:
len(list(generate_perms(parameters)))  # Total number of permutations

4432320

## Optimizing function's hyperparameters

In [8]:
space = {
    'age': hp.choice('age', list(range(14, 71))),
    'height': hp.choice('height', list(range(140, 221))),
    'haircolor': hp.choice('haircolor', ['blonde', 'brown', 'black', 'red', 'gray']),
    'hair': hp.choice('hair', ['long', 'middle', 'short',' bald']),
    'eyecolor': hp.choice('eyecolor', ['blue', 'green', 'brown', 'grey']),
    'beard': hp.choice('beard', ['none', 'small', 'middle', 'large']),
    'body': hp.choice('body', ['muscle', 'normal', 'chubby'])
}

In [9]:
cache = {} # Cache for downloaded scores 

def get_camel_loss(params):
    hashed_params = hash(frozenset(params.items()))
    if hashed_params in cache:
        return cache[hashed_params]
    
    req = generate_request(**params)
    score = -parse_request(req)
    cache[hashed_params] = score
    return score

In [11]:
bayesian_opt_trials = Trials()

# Optimize using Bayesian optimization
best_params = fmin(get_camel_loss, space, algo=tpe.suggest, 
max_evals=10000, trials=bayesian_opt_trials, rstate=np.random.RandomState(2000))

100%|████████████████████████████████████████████████████| 10000/10000 [1:23:24<00:00,  2.00trial/s, best loss: -106.0]


In [16]:
print("Best Results")
print("===================")
print(f"Score is {-bayesian_opt_trials.best_trial['result']['loss']}")
for key, value in best_params.items():
    print(f"{key}: {parameters[key][value]}")

Best Results
Score is 106.0
age: 26
beard: middle
body: muscle
eyecolor: blue
hair: middle
haircolor: blonde
height: 178


In [15]:
cache

{963414367: -59,
 1911357509: -30,
 -1119215189: -39,
 13316853: -46,
 2059649008: -44,
 -708875277: -39,
 1356967811: -27,
 1341586782: -51,
 -2132614403: -32,
 1257935909: -18,
 1541457375: -31,
 -1312320805: -42,
 436115766: -19,
 1331913378: -26,
 -897200854: -51,
 -933872007: -31,
 705176084: -39,
 -1991995711: -51,
 -832239848: -52,
 1875161852: -53,
 2079935688: -60,
 -424530530: -32,
 2063009503: -65,
 954937701: -54,
 1523363981: -48,
 1088378875: -60,
 1609469324: -25,
 -1121462575: -31,
 -1273674758: -57,
 1601684434: -55,
 -226872770: -40,
 -1137911992: -59,
 -611058265: -46,
 1672485002: -46,
 -431208373: -53,
 360694713: -60,
 678183163: -40,
 -1999663070: -36,
 -727692820: -59,
 677001643: -48,
 -6456: -28,
 2005008173: -39,
 -1094060958: -20,
 1173774048: -58,
 48136226: -65,
 -849647710: -59,
 -1438526950: -20,
 995034521: -52,
 2048485821: -64,
 -2049842316: -37,
 -487996158: -60,
 -1189894397: -35,
 1668259286: -56,
 1492836796: -55,
 -223722232: -32,
 726236137: -59